# Effiziente Subset-Funktion
Im Verlauf des Apriori-Algorithmus wird $C_k$ zu $L_k$ transformiert, indem genau die enthaltenen Itemsets entfernt werden, die nicht den vorgegebenen, minimalen Support erfüllen. Dazu ist es nötig, die Häufigkeit jedes Elements in $C_k$ innerhalb aller Transaktionen $T$ zu zählen. In diesem Abschnitt soll eine effizientere Möglichkeit gezeigt werden, um den Support zu bestimmen.

In [ ]:
from tui_dsmt.fpm import numbers, Itemset, HashTree

## Inhaltsverzeichnis
- [Einfache Umsetzung](#Einfache-Umsetzung)
- [Hasbaum zur Speicherung](#Hasbaum-zur-Speicherung)
- [Bestimmung häufiger Itemsets](#Bestimmung-häufiger-Itemsets)

## Einfache Umsetzung
Zunächst soll noch einmal die einfache Umsetzung anhand des bereits bekannten Datensatzes betrachtet werden.

In [ ]:
numbers

Für ein einzelnes Itemset wird dann über alle Transaktionen iteriert und gezählt, ob das Itemset enthalten ist.

In [ ]:
candidate = Itemset(1, 3)
support = 0

for _, items in numbers:
    if candidate.is_subset(items):
        support += 1

support

Diese Schleife muss für jedes Itemset in $C_k$ wiederholt werden, während gleichzeitig eine Transaktion verschiedene Kandidaten enthalten kann. So sind insgesamt $|T| * |C_k|$ Prüfungen notwendig.

## Hasbaum zur Speicherung
Durch eine geschickte Konstruktion eines Hashbaums kann der Aufwand zur Prüfung massiv reduziert werden. Der Hashbaum folgt dabei einigen Prinzipien:
- Die Blattknoten enthalten jeweils eine Liste von Itemsets mit deren Häufigkeiten. (Die Größe der Liste ist begrenzt.)
- Die inneren Knoten enthalten eine Hashtabelle. Jedes Bucket verweist auf Kindknoten des nachfolgenden Levels.
- Die Wurzel befindet sich auf Level 1. Jeder weitere Abstieg erhöht das Level.

Zur Konstruktion des Baums werden alle Kandidaten aus $C_k$ betrachtet und in den Baum eingefügt. Dabei wird auf dem Level $d$ das $d$-te Item des Itemsets entnommen und die Hashfunktion darauf angewendet. Gestartet wird dabei bei der Wurzel und somit dem ersten Element des Itemsets. Wird eine Liste zu lang, wird das Blatt in einen inneren Knoten umgewandelt und die enthaltenen Einträge mit der Hashfunktion neu verteilt.

Sei nun beispielsweise $C_k$ die folgende Menge:

In [ ]:
C_k = {
    (1, 4, 11), (1, 6, 11), (1, 7, 9), (1, 11, 12),
    (2, 3, 8), (2, 4, 6), (2, 4, 7), (2, 5, 6), (2, 5, 7), (2, 7, 9),
    (3, 4, 8), (3, 4, 11), (3, 4, 15), (3, 5, 7), (3, 5, 11), (3, 6, 7), (3, 7, 11),
    (7, 8, 9), (7, 9, 12),
    (5, 6, 7), (5, 7, 10), (5, 8, 11)
}

Die Hash-Funktion $h(K)$ kann beliebig gewählt werden. Im nachfolgenden Beispiel wird $h(K) = K\;mod\;3$ verwendet.

In [ ]:
def hash_function(x):
    return x % 3

Der Aufbau des Hashbaums funktioniert dann wie oben beschrieben:

In [ ]:
ht = HashTree(C_k, hash_function, 3)
ht

## Bestimmung häufiger Itemsets
Innerhalb des entstandenen Baumes werden nun alle Kandidaten gesucht, die in einer Transaktion $T = (t_1, t_2, \dots, t_m)$ enthalten sind. Dazu wird
- bei der Wurzel begonnen und der Hashwert für jedes Item $t_i$ in $T$ bestimmt, um beim resultierenden Kindknoten fortzufahren.
- bei einem inneren Knoten, den man durch Hashing von $t_i$ erreicht hat, wird die Hashfunktion auf alle Items $t_k$ mit $k > i$ angewendet und rekursiv fortgefahren.
- bei einem Blattknoten für alle in der gespeicherten Liste enthaltenen Itemsets getestet, ob sie in der Transaktion $T$ vorkommen.

Auch diesen Teil des Algorithmus können Sie schrittweise nachvollziehen:

In [ ]:
ht.find((1, 3, 7, 9, 12))

Nach dem Durchlauf der Animation verbleiben die in Betracht gezogenen Blätter rot. Im gegebenen Beispiel mussten damit lediglich sechs von $22$ Kandidaten gegen die aktuelle Transaktion geprüft werden.

Soll nun der Support für die Kandidaten innerhalb einer Transaktionsdatenbank gezählt werden, so wird das Vorgehen für jede Transaktion wiederholt. Der Baum muss aber natürlich nur einmal aufgebaut werden. Auch wenn dieses Vorgehen eine separate Datenstruktur benötigt, lässt sich insbesondere bei größeren Transaktionsdatenbanken eine deutliche Reduktion der notwendigen Schritte erreichen.